In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import random
import math
import re
import nltk
nltk.download('punkt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocessing and Splitting

In [3]:
corpus_path = '/content/drive/MyDrive/Auguste_Maquet.txt'

# Load the entire corpus
with open(corpus_path, "r") as f:
    corpus = f.read()

corpus_path = '/content/drive/MyDrive/Auguste_Maquet.txt'

# Load the entire corpus
with open(corpus_path, "r") as f:
    corpus = f.read()

def clean(data):
    data = data.lower()
    data = re.sub(r'\n|\s+', ' ', data)  # replace newline and multiple spaces with single space
    data = re.sub(r"[^a-zA-Z0-9\s,.!?;:]", "", data)
    data = re.sub(r'[’‘]', '\'', data)  # apostrophes
    data = re.sub(r'[“”`\' ]|[–—-]', ' ', data)  # quotes and dashes
    data = data.strip()
    return data

corpus = clean(corpus)
sentences = sent_tokenize(corpus)

def add_start_and_end_tokens(sentences):
  modified_sentences = []
  for sentence in sentences:
    sentence = sentence.lower()
    tokenized_sentence = word_tokenize(sentence)
    new_sentence = ['<S>'] + tokenized_sentence + ['</S>']
    modified_sentences.append(new_sentence)
  return modified_sentences

sentences = add_start_and_end_tokens(sentences)

# Calculate the adjusted split sizes
train_size = int(0.7 * len(sentences))
val_size = int(0.2 * len(sentences))
test_size = int(0.1 * len(sentences))

# Split into train, validation, and test sets
train_sentences = sentences[:train_size]
validation_sentences = sentences[train_size:train_size + val_size]
test_sentences = sentences[train_size + val_size:train_size + val_size + test_size]

# Print the number of tokenized sentences in each set
print(f"Number of tokenized train sentences: {len(train_sentences)}")
print(f"Number of tokenized validation sentences: {len(validation_sentences)}")
print(f"Number of tokenized test sentences: {len(test_sentences)}")

Number of tokenized train sentences: 41528
Number of tokenized validation sentences: 11865
Number of tokenized test sentences: 5932


## Building vocab

In [4]:
from collections import Counter

def build_vocab(sentences):
    all_words = [word for sentence in sentences for word in sentence]
    word_counts = Counter(all_words)
    vocab = set(word_counts.keys())
    return vocab

all_sentences = train_sentences
vocab = build_vocab(all_sentences)

special_tokens = ['<UNK>', '<PAD>', '<S>', '</S>']
for token in special_tokens:
    vocab.add(token)

vocab_size = len(vocab)

print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 22271


## Load Glove embeddings

In [5]:
embedding_dim = 300

def load_glove_embeddings(glove_path):
  word_embeddings = {}
  with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = vector

  special_tokens = ['<UNK>', '<PAD>', '<S>', '</S>']
  for token in special_tokens:
      if token in ['<S>', '</S>']:
      #     # Random initialization for special tokens
          word_embeddings[token] = np.random.rand(embedding_dim).astype('float32')
      elif token == '<UNK>':
          # Mean of all embeddings in GloVe for <UNK>
          all_vectors = np.array(list(word_embeddings.values()))
          mean_vector = np.mean(all_vectors, axis=0)
          word_embeddings[token] = mean_vector
      elif token == '<PAD>':
          # Zero initialization for <PAD>
          word_embeddings[token] = np.zeros(embedding_dim, dtype='float32')


  return word_embeddings

glove_path = '/content/drive/MyDrive/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_path)

# Check embedding dimensions
embedding_dim = len(glove_embeddings[next(iter(glove_embeddings))])
print(f"Loaded GloVe embeddings with dimension: {embedding_dim}")

Loaded GloVe embeddings with dimension: 300


## Create mapping

In [6]:
def build_mappings(vocab, glove_embeddings):
    word2index = {word: idx for idx, word in enumerate(vocab)}
    index2word = {idx: word for word, idx in word2index.items()}

    return word2index, index2word

word2index, index2word = build_mappings(vocab, glove_embeddings)

print(f"Sample word2index mapping: {list(word2index.items())[:10]}")
print(f"Sample index2word mapping: {list(index2word.items())[:10]}")

count = 0;
for word in word2index.keys():
    if word not in glove_embeddings:
        count = count + 1

len(vocab), len(word2index), len(glove_embeddings), count

Sample word2index mapping: [('bosom', 0), ('thistake', 1), ('irongray', 2), ('goodlooking', 3), ('twelvelet', 4), ('mall', 5), ('swept', 6), ('slavemerchants', 7), ('kingdom', 8), ('p.s.be', 9)]
Sample index2word mapping: [(0, 'bosom'), (1, 'thistake'), (2, 'irongray'), (3, 'goodlooking'), (4, 'twelvelet'), (5, 'mall'), (6, 'swept'), (7, 'slavemerchants'), (8, 'kingdom'), (9, 'p.s.be')]


(22271, 22271, 400004, 4631)

In [7]:
pad_idx = word2index['<PAD>']
pad_idx, index2word[pad_idx]

(1766, '<PAD>')

## Embeddings Matrix

In [8]:
embedding_dim = 300
vocab_size = len(word2index)
embeddings_matrix = np.zeros((vocab_size, embedding_dim))

for idx, word in index2word.items():
    if word in glove_embeddings:
        embeddings_matrix[idx] = glove_embeddings[word]
    else:
        unknown_embeds = glove_embeddings['<UNK>']
        embeddings_matrix[idx] = unknown_embeds

print(f"Shape of embeddings matrix: {embeddings_matrix.shape}")

Shape of embeddings matrix: (22271, 300)


## Create Dataset

In [9]:
def get_index(word, word2index):
    return word2index.get(word, word2index.get('<UNK>'))

def create_context_label_dataset(sentences, index2word):
    contexts = []
    labels = []

    for sentence in sentences:
        sentence_indices = [get_index(word, word2index) for word in sentence]

        # Create context (all words except the last) and label (all words except the first)
        context = sentence_indices[:-1]
        label = sentence_indices[1:]

        contexts.append(context)
        labels.append(label)

    return contexts, labels


train_contexts, train_labels = create_context_label_dataset(train_sentences, word2index)
val_contexts, val_labels = create_context_label_dataset(validation_sentences, word2index)
test_contexts, test_labels = create_context_label_dataset(test_sentences, word2index)

In [10]:
for i in range(8):
  print(len(train_contexts[i]), len(train_labels[i]))

39 39
29 29
30 30
179 179
4 4
5 5
4 4
4 4


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class TextDataset(Dataset):
    def __init__(self, contexts, labels):
        self.contexts = contexts
        self.labels = labels

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        return torch.tensor(self.contexts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = TextDataset(train_contexts, train_labels)
validation_dataset = TextDataset(val_contexts, val_labels)
test_dataset = TextDataset(test_contexts, test_labels)

# collate function for padding
def collate_fn(batch):
    contexts, labels = zip(*batch)
    # Pad sequences with '<PAD>' to make all sentences in the batch the same length
    contexts_padded = pad_sequence(contexts, batch_first=True, padding_value=word2index['<PAD>'])
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=word2index['<PAD>'])
    return contexts_padded, labels_padded

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

## LSTM class

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, pad_idx, embedding_matrix, embedding_dim, hidden_dim, output_dim, num_layers=1, dropout=0.4):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), padding_idx=pad_idx, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hidden):
        x = self.embedding(x)

        if hidden is None:
            lstm_out, hidden = self.lstm(x)
        else:
            lstm_out, hidden = self.lstm(x, hidden)

        # Detach hidden state to prevent backward through the graph multiple times
        hidden = (hidden[0].detach(), hidden[1].detach())

        final_out = self.fc(self.dropout(lstm_out))
        return final_out, hidden

# Model params
embedding_dim = 300
hidden_dim = 300
output_dim = len(vocab)
num_layers = 1
dropout = 0.3
pad_idx = word2index['<PAD>']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMModel(pad_idx, embeddings_matrix, embedding_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=0, factor=0.1)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [13]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, num_epochs, patience):
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        # Initialize hidden state for the epoch
        hidden = None

        for batch_idx, (contexts, labels) in enumerate(train_loader):
            contexts, labels = contexts.to(device), labels.to(device)

            optimizer.zero_grad()

            batch_size = contexts.size(0)  # Current batch size
            if hidden is None or hidden[0].size(1) != batch_size:
                hidden = None  # Reset hidden state if batch size is different

            output, hidden = model(contexts, hidden)

            # Reshape output and labels for CrossEntropyLoss
            loss = criterion(output.view(-1, output.size(-1)), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {avg_train_loss:.4f}')

        # Validation step
        model.eval()
        total_val_loss = 0
        hidden = None

        with torch.no_grad():  # Disable gradient calculation for validation
            for batch_idx, (contexts, labels) in enumerate(val_loader):
                contexts, labels = contexts.to(device), labels.to(device)

                batch_size = contexts.size(0)
                if hidden is None or hidden[0].size(1) != batch_size:
                    hidden = None

                # Forward pass through the model
                output, hidden = model(contexts, hidden)

                # Compute the validation loss
                loss = criterion(output.view(-1, output.size(-1)), labels.view(-1))
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f'Validation Loss: {avg_val_loss:.4f}')
        scheduler.step(avg_val_loss)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'LSTM_model.pth')
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break


In [15]:
train_model(model, train_loader, validation_loader, criterion, optimizer, scheduler, device, num_epochs = 10, patience = 3)

Epoch 1/10
Train Loss: 5.6287
Validation Loss: 5.1324
Epoch 2/10
Train Loss: 4.9682
Validation Loss: 4.8892
Epoch 3/10
Train Loss: 4.7369
Validation Loss: 4.7650
Epoch 4/10
Train Loss: 4.5758
Validation Loss: 4.6875
Epoch 5/10
Train Loss: 4.4456
Validation Loss: 4.6348
Epoch 6/10
Train Loss: 4.3364
Validation Loss: 4.6055
Epoch 7/10
Train Loss: 4.2404
Validation Loss: 4.5811
Epoch 8/10
Train Loss: 4.1553
Validation Loss: 4.5690
Epoch 9/10
Train Loss: 4.0784
Validation Loss: 4.5699
Epoch 10/10
Train Loss: 3.9619
Validation Loss: 4.5676


## Evaluate model

In [18]:
def evaluate_model(model, test_loader, device, criterion):
      model.eval()  # Set model to evaluation mode
      total_test_loss = 0
      hidden = None

      with torch.no_grad():  # Disable gradient calculation for validation
          for batch_idx, (contexts, labels) in enumerate(test_loader):
              contexts, labels = contexts.to(device), labels.to(device)

              batch_size = contexts.size(0)
              if hidden is None or hidden[0].size(1) != batch_size:
                  hidden = None  # Reset hidden state if batch size is different

              # Forward pass through the model
              output, hidden = model(contexts, hidden)

              # Compute the validation loss
              loss = criterion(output.view(-1, output.size(-1)), labels.view(-1))
              total_test_loss += loss.item()

      avg_test_loss = total_test_loss / len(test_loader)

    # Calculate perplexity
      perplexity = math.exp(avg_test_loss)

      return avg_test_loss, perplexity

In [19]:
train_loss, train_perplexity = evaluate_model(model, train_loader, device, criterion)
val_loss, val_perplexity = evaluate_model(model, validation_loader, device, criterion)
test_loss, test_perplexity = evaluate_model(model, test_loader, device, criterion)

In [20]:
print(f"Train Loss: {train_loss:.4f}")
print(f"Train Perplexity: {train_perplexity:.4f}")

Train Loss: 3.8203
Train Perplexity: 45.6201


In [21]:
print(f"Val Loss: {val_loss:.4f}")
print(f"Val Perplexity: {val_perplexity:.4f}")

Val Loss: 4.5661
Val Perplexity: 96.1638


In [22]:
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Perplexity: {test_perplexity:.4f}")

Test Loss: 4.5510
Test Perplexity: 94.7288


In [23]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

save_model(model,"2021101102-LM2-LSTM.pth")

Model saved to 2021101102-LM2-LSTM.pth


## Saving perplexities to file

In [26]:
def save_perplexities(model, test_loader, file_name):

    model.eval()
    batch_perplexities = []
    total_test_loss = 0
    hidden = None

    with torch.no_grad():
        for batch_idx, (contexts, labels) in enumerate(test_loader):
            contexts, labels = contexts.to(device), labels.to(device)

            batch_size = contexts.size(0)
            if hidden is None or hidden[0].size(1) != batch_size:
                hidden = None  # Reset hidden state if batch size is different

            # Forward pass through the model
            output, hidden = model(contexts, hidden)

            # Compute the validation loss
            loss = criterion(output.view(-1, output.size(-1)), labels.view(-1))
            total_test_loss += loss.item()

    # Compute the average test loss and perplexity
    avg_test_loss = total_test_loss / len(test_loader)
    avg_perplexity = math.exp(avg_test_loss)

    with open(file_name, 'a') as f:
        f.write(f'Average Perplexity: {avg_perplexity:.4f}\n')

    # Print average perplexity
    print(f'Average Perplexity: {avg_perplexity:.4f}')

In [27]:
train_path = "2021101102-LM2-train-perplexity.txt"
val_path = "2021101102-LM2-val-perplexity.txt"
test_path = "2021101102_LM2-test-perplexity.txt"

# Createa loaders with size 1
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

save_perplexities(model, train_loader, train_path)
save_perplexities(model, validation_loader, val_path)
save_perplexities(model, test_loader, test_path)

Average Perplexity: 37.0503
Average Perplexity: 74.7442
Average Perplexity: 73.0540
